In [18]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [4]:
design.get_chip_z('Q_chip')

'6um'

In [20]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='6um'

# Resonator length

In [21]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [22]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


# Flipchip

In [23]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [24]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [25]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [26]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [27]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [28]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [29]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [30]:
hfss.start()

INFO 12:27PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:27PM [load_ansys_project]: 	Opened Ansys App
INFO 12:27PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 12:27PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project1
INFO 12:27PM [connect_design]: No active design found (or error getting active design).
INFO 12:27PM [connect]: 	 Connected to project "Project1". No design detected


True

In [31]:
hfss.activate_ansys_design("flip6um_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

12:28PM 19s WARNING [activate_ansys_design]: The design_name=flip6um_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 12:28PM [connect_design]: 	Opened active design
	Design:    flip6um_V1 [Solution type: Eigenmode]
WARNING 12:28PM [connect_setup]: 	No design setup detected.
WARNING 12:28PM [connect_setup]: 	Creating eigenmode default setup.
INFO 12:28PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [32]:
hfss.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.98',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.98',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [15]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:18AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpy_4e0o71.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4980000000, Maxwell, 1, False
INFO 10:18AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpokem4ziy.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4980000000, Maxwell, 1, False
INFO 10:18AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmppn58ae6b.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4980000000, Maxwell, 2, False
INFO 10:18AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp93hq9jwb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4980000000, Maxwell, 3, False
INFO 10:18AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmptfqsef6w.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4980000000, Maxwell, 4, False
INFO 10:18

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,395.530,-132.04000,-130.330,-67.34200
pad_bot_Q0,-132.040,160.94000,-28.259,-0.23645
pad_top_Q0,-130.330,-28.25900,161.350,-2.48600
quad_coupler_c0,-67.342,-0.23645,-2.486,70.29100


In [16]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.55
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.674188 [GHz]
EC 200.661549 [MHz]
EJ 14.854145 [GHz]
alpha -223.732240 [MHz]
dispersion 0.174410 [KHz]
Lq 10.995564 [nH]
Cq 96.531838 [fF]
T1 15776.118036 [us]

**Coupling Properties**

tCqbus1 -1.122004 [fF]
gbus1_in_MHz -13.746145 [MHz]
χ_bus1 -0.010062 [MHz]
1/T1bus1 10.088346 [Hz]
T1bus1 15776.118036 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.552953,288.670493,14.854145,-330.726105,8.836791,[-57.1599770871709],[-0.477807327857014],0.477807,57.159977
2,4.799674,212.145659,14.854145,-237.412473,0.334021,[-7.531177356540906],[-0.00345746739312933],0.003457,7.531177
3,4.798697,212.054872,14.854145,-237.303992,0.332379,[-10.962086346296054],[-0.00731741722341181],0.007317,10.962086
4,4.709224,203.832383,14.854145,-227.500959,0.209843,[-13.347677153786789],[-0.009849798747533054],0.009850,13.347677
5,4.684411,201.583951,14.854145,-224.8279,0.184134,[-12.75270010192615],[-0.008755301426196387],0.008755,12.752700
6,4.688396,201.944118,14.854145,-225.255867,0.188058,[-12.848345338726736],[-0.00892511010575359],0.008925,12.848345
7,4.674188,200.661549,14.854145,-223.73224,0.17441,[-13.746144647585062],[-0.01006190998613684],0.010062,13.746145


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
